<a href="https://colab.research.google.com/github/Ravinesh85/testrepo1/blob/master/Document_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!pip install -U -q google-generativeai




In [15]:
import textwrap
import numpy as np
import pandas as pd

import google.generativeai as genai

# Used to securely store your API key
from google.colab import userdata

from IPython.display import Markdown




In [16]:
API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=API_KEY)



In [17]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


In [18]:
DOCUMENT1 = {
    "title": "Asset details latest",
    "content": "This document contains details of the latest asset owned by CAT Dealers & Customers. Sample assets are Drilling machine 5 W , Power appliances ."}
DOCUMENT2 = {
    "title": "False Pipeline Alert ",
    "content": "This is a false pipeline alert. Pipeline is stable. Please close this incidnet without taking action"}
DOCUMENT3 = {
    "title": "Fluid sample",
    "content": "This document conatins information about Fluid sample data object. Fluid sample is a RDS table and can be accessed via pgadmin"}

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

In [19]:
# Get the embeddings of each text and add to an embeddings column in the dataframe
df = pd.DataFrame(documents)
df.columns = ['Title', 'Text']
df

,Title,Text
0,Asset details latest,This document contains details of the latest a...
1,False Pipeline Alert,This is a false pipeline alert. Pipeline is st...
2,Fluid sample,This document conatins information about Fluid...


In [23]:
def embed_fn(title, text):
  return genai.embed_content(model="models/embedding-001",
                             content=text,
                             task_type="retrieval_document",
                             title=title)["embedding"]

df['Embeddings'] = df.apply(lambda row: embed_fn(row['Title'], row['Text']), axis=1)
df

,Title,Text,Embeddings
0,Asset details latest,This document contains details of the latest a...,"[0.033577252, -0.0108568575, -0.041498434, -0...."
1,False Pipeline Alert,This is a false pipeline alert. Pipeline is st...,"[0.048597097, -0.06663983, -0.031086525, -0.04..."
2,Fluid sample,This document conatins information about Fluid...,"[0.0010982918, -0.039055478, -0.035472173, 0.0..."


In [24]:
query = " False Alert"
model = 'models/embedding-001'

request = genai.embed_content(model=model,
                              content=query,
                              task_type="retrieval_query")

In [25]:
def find_best_passage(query, dataframe):
  """
  Compute the distances between the query and each document in the dataframe
  using the dot product.
  """
  query_embedding = genai.embed_content(model=model,
                                        content=query,
                                        task_type="retrieval_query")
  dot_products = np.dot(np.stack(dataframe['Embeddings']), query_embedding["embedding"])
  idx = np.argmax(dot_products)
  return dataframe.iloc[idx]['Text'] # Return text from index with max value


In [26]:
passage = find_best_passage(query, df)
passage

'This is a false pipeline alert. Pipeline is stable. Please close this incidnet without taking action'